# BASKETBALL SELECTION PROBLEM

We are going to develop a **QRBS** for selecting a basket player. The inputs of this **QRBS** will be the *height* of the player and the *throw skill of the player* that is evaluated by the number of annonated throws over 20.

In [ ]:
import sys
sys.path.append("../")
import itertools as it
import pandas as pd
import numpy as np

In [ ]:
from neasqc_qrbs.qrbs import QRBS
from neasqc_qrbs.knowledge_rep import AndOperator, OrOperator, NotOperator
from selectable_qpu import SelectableQPU

qpu_selected = SelectableQPU()

In [ ]:
# myQLM qpus
from qpu.select_qpu import select_qpu
# List with the strings that should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]
qpu_config_c = {
    "qpu_type": ideal_qpus[0], 
}
qpu = select_qpu(qpu_config_c)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

## 0. Function for selecting the membership function of the different facts

We are going to define the two following functions:

* **degree_of_membership**: This functions computes the degree of membership of a fixed input **x** to a picewise linear function defined by an input **text** string in the following format: **"y_0/x_0-y_1/x_1-...-y_f/x_f"**.

* **get_membership_function**: this functions returns the linear picewise membership_function for a input **domain**. The picewise definition should be provided in the following format: **"y_0/x_0-y_1/x_1-...-y_f/x_f"**

The idea is use this functions for asigning the precision of the facts in function of the inputs 

In [ ]:
def degree_of_membership(x, text):
    list_ = text.split("-")      
    
    for i in range(len(list_)-1):
        str_start = list_[i]
        x_0 = float(str_start.split("/")[1])
        y_0 = float(str_start.split("/")[0])
        
        if (i==0) and (x <= x_0):
            return y_0
        str_end = list_[i+1]        
        x_1 = float(str_end.split("/")[1])
        y_1 = float(str_end.split("/")[0])           
        if (x >= x_0) and (x <= x_1):
            slope = (y_1 - y_0) / (x_1 - x_0)
            intercept = y_1 - slope * x_1
            return slope * x + intercept
    return y_1
def get_membership_function(domain, text):

    list_ = text.split("-")
    domain_x_0 = float(list_[0].split("/")[1])
    domain_x_f = float(list_[-1].split("/")[1])
    array = np.array(
        list(map(lambda x: degree_of_membership(x, text), domain))
    )
    #array = np.where(np.array(domain) >= domain_x_0, array, np.NaN)
    #array = np.where(np.array(domain) <= domain_x_f, array, np.NaN)
    return array

Here we provide some example of uses:

In [ ]:
# Definition of the mebership function
definition_mf = "0/11-1/13-1/15-0/17"

# Getting the degree of membership for x
x = 16
print("For x= {} the degree of membership to the funcion: {} is {}".format(
    x, definition_mf, degree_of_membership(x, definition_mf)))

# Getting the membership function for an input domain

domain = np.linspace(0, 20, 100)
m_f= get_membership_function(domain, definition_mf)
plt.plot(domain, m_f)

In [ ]:
# Tests 
# MUST be 0
y = degree_of_membership(2, "0/11-1/13-1/15-0/17")
print("MUST be 0. Result: ", y) 

# Between 0 - 1
y = degree_of_membership(12, "0/11-1/13-1/15-0/17")
print("BETWEEN 0-1. Result: ", y) 

# # MUST be 1
y = degree_of_membership(13, "0/11-1/13-1/15-0/17")
print("MUST be 1. Result: ", y) 

# # MUST be 1
y = degree_of_membership(14, "0/11-1/13-1/15-0/17")
print("MUST be 1. Result: ", y) 

# Between 0 - 1
y = degree_of_membership(16, "0/11-1/13-1/15-0/17")
print("BETWEEN 0-1. Result: ", y) 

# MUST be 0
y = degree_of_membership(17, "0/11-1/13-1/15-0/17")
print("MUST be 0. Result: ", y) 

## 1. Create the different parts of the QRBS

First, as usual, we need to instantiate the **Quantum Rule Based System**

In [ ]:
# Instantiate the QRBS
basket =  QRBS()

### 1.1 Facts

Now we need to define the different facts (inputs and outputs) for the **QRBS**:
* Input facts:
    * Related to the throw skill of the player
    * Related to the height of the player
* Output facts: The final evaluation of the player. It depends on the rule of the **QRBS** and the input facts


#### Related to the throw skill of the player

Here we program the *facts* that deal with the throws of the player. We are going to create 4 possible classifications for the throw skill of the player depending on the number of hitting throws in 20 throws. We define the following membership functions for each of the 4 possible classifications:

* "1/0-1/3-0/7" -> throw_very_bad
* "0/3-1/5-1/7-0/9" -> throw_bad
* "0/7-1/10-0/13" -> throw_regular
* "0/11-1/13-1/15-0/17" -> throw_good
* "0/15-1/17-1/20" -> throw_very_good

Now we can create the corresponding facts based on this classification

In [ ]:
# Free throw scoring

throw_very_bad = basket.assert_fact("throw_very_bad", "1/0-1/3-0/7")
throw_bad = basket.assert_fact("throw_bad", "0/3-1/5-1/7-0/9")
throw_regular = basket.assert_fact("throw_regular", "0/7-1/10-0/13")
throw_good = basket.assert_fact("throw_good", "0/11-1/13-1/15-0/17")
throw_very_good = basket.assert_fact("throw_very_good", "0/15-1/17-1/20")

The **get_membership_function** can be used for getting the corresponding membership function for each possible fact related with the throw skill

In [ ]:
domain_throw = np.array(range(0, 21))
m_f = get_membership_function(domain_throw, throw_bad.value)
plt.plot(domain_throw, m_f)

In [ ]:
# We can plot all the membership functions for all the facts
for fact in [throw_very_bad, throw_bad, throw_regular, throw_good, throw_very_good]:
    plt.plot(
        domain_throw,
        get_membership_function(domain_throw, fact.value)
    )
plt.xlabel("Throws (of 20)")
plt.xlim(0, 20)
plt.ylabel("Scoring throw")
plt.ylim(0, 1.1)
plt.legend(["throw_very_bad", "throw_bad", "throw_regular", "throw_good", "throw_very_good"])

#### Related to the height of the player

For dealing with the height of the player we are going to define 5 classifications with the following membership function definitions:

* *height_very_small*: "1/150-1/170-0/180"
* *height_small*: "0/170-1/175-1/180-0/185"
* *height_normal*: "0/180-1/190-0/195"
* *height_tall*: "0/190-1/195-1/205-0/210"
* *height_very_tall*: "0/200-1/210-1/250"

Now we can define the facts related to the height:

In [ ]:
# height scoring

height_very_small = basket.assert_fact("height_very_small", "1/150-1/170-0/180")
height_small = basket.assert_fact("height_small", "0/170-1/175-1/180-0/185")
height_normal = basket.assert_fact("height_normal", "0/180-1/190-0/195")
height_tall = basket.assert_fact("height_tall", "0/190-1/195-1/205-0/210")
height_very_tall = basket.assert_fact("height_very_tall", "0/200-1/210-1/250")

The **get_membership_function** can be used for getting the corresponding membership function for each possible fact!!

In [ ]:
# We can plot all the membership functions for all the facts
domain_height = np.array(range(150, 250, 5))

for fact in [height_very_small, height_small, height_normal, height_tall, height_very_tall]:
    plt.plot(
        domain_height,
        get_membership_function(domain_height, fact.value)
    )
    
plt.xlabel("Height (cm)")
plt.xlim(150, 250)
plt.ylabel("Scoring height")
plt.ylim(0, 1.05)    
plt.legend(["height_very_small", "height_small", "height_normal", "height_tall", "height_very_tall"])

#### Output facts

Finally, we are going to classify the player into 4 classifications with the following membership definition functions.

* player_bad: "1/0-0/1"
* player_normal: "1/0-1/25-0/40"
* player_good: "0/25-1/40-1/60-0/75"
* player_very_good: "0/60-1/75-1/100"

In [ ]:
# player score

player_bad = basket.assert_fact("player_bad", "1/0-0/1")
player_normal = basket.assert_fact("player_normal", "1/0-1/25-0/40")
player_good = basket.assert_fact("player_good", "0/25-1/40-1/60-0/75")
player_very_good = basket.assert_fact("player_very_good", "0/60-1/75-1/100")

We can use the *get_function* for getting the corresponding score classifications!

In [ ]:
domain_score = np.array(range(0, 101))
for i in [player_bad, player_normal, player_good, player_very_good]:
    plt.plot(
        domain_score,
        get_membership_function(domain_score, i.value)
    )
plt.xlabel("Output Score")
plt.xlim(0, 100)
plt.ylabel("Scoring")
plt.ylim(0, 1.05)     
plt.legend(["player_bad", "player_normal", "player_good", "player_very_good"])    

### 1.2 Rules

Now we need to create the rules of our **QRBS**. We need to create rules that fill each one of the 4 output score classifications, i.e., we need to define rules for classify a player as:

* player_bad: rules for a bad player
* player_normal: rules for a normal player
* player_good: rules for a good player
* player_very_good: rules for a very good player

#### Rules for bad players.

Rules for bad players. 

* Throw: very bad **OR** Throw: bad. In this case, we create the fact: *player_bad_throw* for specifying that the player is bad due to their throw performance.
* Height: very small **OR** Height: small. In this case, we create the fact: *small_player* or specify that the player is not enough tall for be a basketball player.
* Height: normal **OR** Throw: regular. In this case, we create the fact: *normal_regular* to specify that the player is mediocre.


In [ ]:
# Rules for Bad player

# Bad thrown perfomance
throw_bad_player = basket.assert_fact("player_bad_throw", "player with bad throw")
rule_4 = basket.assert_rule(
    OrOperator(throw_very_bad, throw_bad),
    throw_bad_player,
    1.0
)

# Not enough tall player
small_player = basket.assert_fact("small_player", "player too small")
rule_5 = basket.assert_rule(
    OrOperator(height_very_small, height_small),
    small_player,
    1.0
)

# mediocre player
normal_regular = basket.assert_fact("normal_regular", "Normal height and regular throw")
rule_6 = basket.assert_rule(
    AndOperator(height_normal, throw_regular),
    normal_regular,
    1.0
)

# Rule for bad player
rule_bad_player = basket.assert_rule(
    OrOperator(
        OrOperator(throw_bad_player, small_player),
        normal_regular
    ),
    player_bad,
    1.0
)    

#### Rules for normal players.

The following rules define a normal player:

* Height: normal **AND** Throw: good.
* Height: normal **AND** Throw: very good.
* Height: tall **AND** Throw: regular.
* Height: very tall **AND** Throw: regular.

In [ ]:
# Rule for normal player

normal_0 = AndOperator(height_normal, throw_good)
normal_1 = AndOperator(height_normal, throw_very_good)
normal_2 = AndOperator(height_tall, throw_regular)
normal_3 = AndOperator(height_very_tall, throw_regular)

normal_f = OrOperator(
    OrOperator(
        OrOperator(normal_0, normal_1),
        normal_2),
    normal_3)

rule_1 = basket.assert_rule(normal_f, player_normal, 1.0)

#### Rules for good players:

The following rules define a good player:

* Height: tall **AND** Throw: good.
* Height: very tall **AND** Throw: good.


In [ ]:
# Rule for good player

rule_2 = basket.assert_rule(
    OrOperator(
        AndOperator(height_tall, throw_good),
        AndOperator(height_very_tall, throw_good)
    ),
    player_good,
    1.0
)

#### Rules for a very good player

* Height: tall **AND** Throw: very good.
* Height: very tall **AND** Throw: very good.

In [ ]:
# Rule for very good player

rule_3 = basket.assert_rule(
    OrOperator(
        AndOperator(height_tall, throw_very_good),
        AndOperator(height_very_tall, throw_very_good)
    ),
    player_very_good,
    1.0
) 

### 1.3 Islands

Finally, we need to define the *knowledge islands*. One island for output.

In [ ]:
island_normal = basket.assert_island([rule_1])
island_good = basket.assert_island([rule_2])
island_very_good = basket.assert_island([rule_3])
# Bad player has several chained rules
island_bad = basket.assert_island([rule_4, rule_5, rule_6, rule_bad_player])

### 1.4 Inference procces

Now we have the complete definition of the **QRBS** so we can execute some computations:

In [ ]:
target_throws = 16 # of 20
height_player = 198 #cm

In [ ]:
# Asigning the precision of the facts

# for throw facts
list_throws = [throw_very_bad, throw_bad, throw_regular, throw_good, throw_very_good]
for fact_th in list_throws:
    fact_th.precision = degree_of_membership(target_throws, fact_th.value)
    
# for heihgt facts
list_height = [height_very_small, height_small, height_normal, height_tall, height_very_tall]
for fact_he in list_height:
    fact_he.precision = degree_of_membership(height_player, fact_he.value)

In [ ]:
# Precision for the throw facts
[f.precision for f in list_throws]

In [ ]:
# Precision for the height facts
[f.precision for f in list_height]

Now we can execute the inference process!! 
We need to provide:
* qpu
* shots
* model for indetermination propagation

In [ ]:
qpu_selected.execute(basket, qpu=qpu, shots=100, model="cf")

Now we have the precision for each of the output facts!

In [ ]:
player_score = [player_bad, player_normal, player_good, player_very_good]

In [ ]:
[i.precision for i in player_score]

Additionally we can have the precisions for the facts for bad player:

In [ ]:
[f_.precision for f_ in [throw_bad_player, small_player, normal_regular]]

### 1.5 Output computation

The idea is for a player to provide a score between [0-100] based on the scores for each of the precision of the output facts.

For each possible output fact, we need to get the degree of membership of the player (this is the computed precision of the corresponding output fact) to each of the defined membership functions

In [ ]:
# Degree of membership of the player to the desired output_fact
score_domain = np.array(range(0,101))
output_fact = player_good

# first we create the membership function over the complete domain
membership_function = get_membership_function(score_domain, output_fact.value)
# Now we select the precision over the whole domain 
player_output = np.where(membership_function < output_fact.precision, membership_function, output_fact.precision)
player_output = np.where(np.isnan(membership_function), np.nan, player_output)

plt.plot(score_domain, membership_function)
plt.plot(score_domain, player_output, 'o')

plt.legend([
    "Membership function: {}".format(output_fact.attribute),
    "Player Degree of membership for function: {}".format(output_fact.attribute),    
])


In [ ]:
score_domain = np.array(range(0,101))

degree_of_memberships = []
for output_fact in [player_bad, player_normal, player_good, player_very_good]:
    membership_function = get_membership_function(score_domain, output_fact.value)
    player_output = np.where(membership_function < output_fact.precision, membership_function, output_fact.precision)
    player_output = np.where(np.isnan(membership_function), np.nan, player_output)
    degree_of_memberships.append(player_output)

Now we have the degree of membership of the player for the different outputs.

In [ ]:
legends = []
for pmf, o_f in zip(degree_of_memberships,[player_bad, player_normal, player_good, player_very_good]):
    plt.plot(score_domain, pmf)
    legends.append([o_f.attribute])
plt.legend(legends)    

In [ ]:
# Aggregation of the rules
z = np.nanmax(np.array(degree_of_memberships), axis=0)

In [ ]:
plt.plot(
    domain_score, z, 
)

In [ ]:
final_score = np.sum(z * domain_score) / np.sum(z)

In [ ]:
print("For target_throws: {} and height_player: {} the final_score is: {}".format(
    target_throws, height_player, final_score)
)

#### Quantum Circuits

We can visualize the different quantum circuits of each *knowledge island* in the different indetermination models.

In [ ]:
# The three Builders
from neasqc_qrbs.knowledge_rep import  BuilderImpl, BuilderBayes, BuilderFuzzy

In [ ]:
builder = BuilderFuzzy

In [ ]:
c = island_normal.build(builder)
%qatdisplay c --svg

In [ ]:
c = island_good.build(builder)
%qatdisplay c --svg

In [ ]:
c = island_very_good.build(builder)
%qatdisplay c --svg

In [ ]:
c = island_bad.build(builder)
%qatdisplay c --svg

## 2. Complete system

Now we can program the complete system as a function and we only provide the throw and the height of the player for getting a score

In [ ]:
def basquet_qrbs(throw, height, qpu, type_qpu=None, shots=None, model='cf'):
    # Instantiate the QRBS
    basket =  QRBS()
    
    
    #### Initial Facts ######
    
    # Free throw scoring
    throw_very_bad = basket.assert_fact("throw_very_bad", "1/0-1/3-0/7")
    throw_bad = basket.assert_fact("throw_bad", "0/3-1/5-1/7-0/9")
    throw_regular = basket.assert_fact("throw_regular", "0/7-1/10-0/13")
    throw_good = basket.assert_fact("throw_good", "0/11-1/13-1/15-0/17")
    throw_very_good = basket.assert_fact("throw_very_good", "0/15-1/17-1/20")
    
    # height scoring
    height_very_small = basket.assert_fact("height_very_small", "1/150-1/170-0/180")
    height_small = basket.assert_fact("height_small", "0/170-1/175-1/180-0/185")
    height_normal = basket.assert_fact("height_normal", "0/180-1/190-0/195")
    height_tall = basket.assert_fact("height_tall", "0/190-1/195-1/205-0/210")
    height_very_tall = basket.assert_fact("height_very_tall", "0/200-1/210-1/250")  
    
    # player score
    player_bad = basket.assert_fact("player_bad", "1/0-0/1")
    player_normal = basket.assert_fact("player_normal", "1/0-1/25-0/40")
    player_good = basket.assert_fact("player_good", "0/25-1/40-1/60-0/75")
    player_very_good = basket.assert_fact("player_very_good", "0/60-1/75-1/100")   
    
    output = [player_bad, player_normal, player_good, player_very_good]
    
         
    ###### RULES ######
    
    rule_certainty = 0.8
    
    # Rule for normal player
    normal_0 = AndOperator(height_normal, throw_good)
    normal_1 = AndOperator(height_normal, throw_very_good)
    normal_2 = AndOperator(height_tall, throw_regular)
    normal_3 = AndOperator(height_very_tall, throw_regular)
    normal_f = OrOperator(
        OrOperator(
            OrOperator(normal_0, normal_1),
            normal_2),
        normal_3)

    rule_1 = basket.assert_rule(normal_f, player_normal, rule_certainty)    
    
    # Rule for good player
    rule_2 = basket.assert_rule(
        OrOperator(
            AndOperator(height_tall, throw_good),
            AndOperator(height_very_tall, throw_good)
        ),
        player_good,
        rule_certainty
    )    
    # Rule for very good player
    rule_3 = basket.assert_rule(
        OrOperator(
            AndOperator(height_tall, throw_very_good),
            AndOperator(height_very_tall, throw_very_good)
        ),
        player_very_good,
        rule_certainty
    )     
    
    # Rules for Bad player

    throw_bad_player = basket.assert_fact("player_bad_throw", "player with bad throw")
    rule_4 = basket.assert_rule(
        OrOperator(throw_very_bad, throw_bad),
        throw_bad_player,
        rule_certainty
    )

    small_player = basket.assert_fact("small_player", "player too small")
    rule_5 = basket.assert_rule(
        OrOperator(height_very_small, height_small),
        small_player,
        rule_certainty
    )

    normal_regular = basket.assert_fact("normal_regular", "Normal height and regular throw")
    rule_6 = basket.assert_rule(
        AndOperator(height_normal, throw_regular),
        normal_regular,
        rule_certainty
    )

    rule_bad_player = basket.assert_rule(
        OrOperator(
            OrOperator(throw_bad_player, small_player),
            normal_regular
        ),
        player_bad,
        rule_certainty
    ) 
    
    
    ####### KNOWLEDGE iSLANDS ################
    
    island_normal = basket.assert_island([rule_1])
    island_good = basket.assert_island([rule_2])
    island_very_good = basket.assert_island([rule_3])
    island_bad = basket.assert_island([rule_4, rule_5, rule_6, rule_bad_player])   
    
    
    ####### LOADING DATA #########################
    
    
    # Score data
    list_score = [throw_very_bad, throw_bad, throw_regular, throw_good, throw_very_good]
    # print([fact.precision for fact in list_score])
    
    
    for fact in list_score:
        fact.precision = degree_of_membership(throw, fact.value)      
    # print([fact.precision for fact in list_score])
    
    
    # Height data
    # Asigning the precision of the facts
    list_height = [height_very_small, height_small, height_normal, height_tall, height_very_tall]
    # print([fact.precision for fact in list_height])
    for fact in list_height:
        fact.precision = degree_of_membership(height, fact.value)
    # print([fact.precision for fact in list_height])
        
    # Inference Execution
    qpu.execute(basket, qpu=type_qpu, shots=shots, model=model)
    
    # Output post processing
    #output_precision = [fact.precision for fact in output]
    additional_info = [throw_bad_player, small_player, normal_regular]
    
    score_domain = np.array(range(0,101))

    # Computing degree of membership for each output fact for the input player
    degree_of_memberships = []
    for output_fact in output:
        membership_function = get_membership_function(score_domain, output_fact.value)
        player_output = np.where(membership_function < output_fact.precision, membership_function, output_fact.precision)
        player_output = np.where(np.isnan(membership_function), np.nan, player_output)
        degree_of_memberships.append(player_output)    
    # Aggregation of the rules
    z = np.nanmax(np.array(degree_of_memberships), axis=0)
    # Final score
    final_score = np.sum(z * score_domain) / np.sum(z)
    
    
    output_dict = {
        "output_facts" : output,
        "additional_facts" : additional_info,
        "degree_of_memberships": degree_of_memberships,
        "z": z,
        "final_score":final_score
    }
    
    return output_dict

In [ ]:
target_throws = 16 # of 20
height_player = 198 #cm
shots = 0 
model = "cf"

score = basquet_qrbs(
    target_throws, 
    height_player,
    qpu_selected,
    type_qpu=qpu,
    shots=shots,
    model=model
)

In [ ]:
score['output_facts'][0].attribute

In [ ]:
# Degree of mebership of the player to the different output facts
for fact in score["output_facts"]:
    print(fact.attribute, ": ",  str(fact.precision))

In [ ]:
# Score function of the player
plt.plot(score["z"])
plt.xlabel("final score")
plt.ylabel("Z function")

In [ ]:
print("For target_throws: {} and height_player: {} the final_score is: {}".format(
    target_throws, height_player, score["final_score"])
)

#### Some testing

We are going to test the implementation for different fixed inputs.

In [ ]:
def get_pdf_facts(input_dict):

    fact_dict = []
    for p in input_dict:
        fact_dict.append([r.precision for r in p["output_facts"]])
    facts_ = pd.DataFrame(
        fact_dict,
        columns=[i.attribute for i in input_dict[0]["output_facts"] ]
    )

    fact_dict = []
    for p in input_dict:
        fact_dict.append([r.precision for r in p["additional_facts"]])

    add_facts_ = pd.DataFrame(
        fact_dict,
        columns=[i.attribute for i in input_dict[0]["additional_facts"] ]
    )   

    pdf = pd.concat([facts_, add_facts_], axis=1)
    return pdf


#### Input Bad player: bad shooter

Bad shooters will have lees than 8 target throws!!

For all these cases the **player_bad** should have a precision near to 1.0. Additionally, the additional fact **player_bad_throw** should have a precision near 1.0

In [ ]:
# Bad player because of throws

#domain = range(6,9)
#domain = range(0,5)
domain = range(0,8)

bad_player_bad_throws= [basquet_qrbs(
    i, 210, qpu_selected, type_qpu=qpu, shots=0, model='cf') for i in domain]

pdf = get_pdf_facts(bad_player_bad_throws)
pdf["Throws"] = domain
pdf

#### Input Bad player: not tall enough

players with heights lower than 180 are bad players too.

For this case, the fact *player_bad* should have a precision near to 1.0. The additional fact *small_player* should have a precision near to 1.0

In [ ]:
# Bad player because of height

#domain = [150, 160, 170, 175, 180]
domain = range(150, 180, 1)
#domain = range(176, 185, 2)

bad_player_not_tall = [
    basquet_qrbs(20, i, qpu_selected, type_qpu=qpu, shots=0, model='cf') 
    for i in domain
]

pdf = get_pdf_facts(bad_player_not_tall)
pdf["Throws"] = domain
pdf

In [ ]:
# Normal Player

throws = [10, 10, 13, 17]
heights = [195, 210, 190, 190]


normal_player = [
    basquet_qrbs(t, h, qpu_selected, type_qpu=qpu, shots=0, model='fuzzy') for t, h in zip(throws, heights)
]

pdf = get_pdf_facts(normal_player)

pdf["Throws"] = throws
pdf["Heights"] = heights
pdf

In [ ]:
# Good Player


throws = [13, 14, 15]
heights = [195, 200, 205]

good_player = [
    basquet_qrbs(t, h, qpu_selected, type_qpu=qpu, shots=0, model='fuzzy') for t, h in zip(throws, heights)
]

pdf = get_pdf_facts(good_player)

pdf["Throws"] = throws
pdf["Heights"] = heights
pdf


In [ ]:
# Very Good Player

throws = [17, 18, 16]
heights = [195, 205, 210]


v_good_player = [
    basquet_qrbs(t, h, qpu_selected, type_qpu=qpu, shots=0, model='fuzzy') for t, h in zip(throws, heights)
]

pdf = get_pdf_facts(v_good_player)

pdf["Throws"] = throws
pdf["Heights"] = heights
pdf

### Player Evaluation Time!!

In [ ]:
Name = ["Elias", "Blas", "Luis", "Juan", "Raul", "Cholo", "Zalo"]
Throws = [16, 17, 17, 15, 18, 18, 18]
Heights = [198, 193, 188, 203, 176, 186, 200]

In [ ]:
player_evaluation = [
    basquet_qrbs(t, h, qpu_selected, type_qpu=qpu, shots=0, model='cf') for t, h in zip(Throws, Heights)
]
list_ = [[n, t, h, p["final_score"]] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation)]

pdf = pd.DataFrame(
    list_,
    columns = ["Name", "Throws", "Height", "Final_Score"]
)
pdf.sort_values(["Final_Score"], ascending=False)

In [ ]:
player_evaluation = [
    basquet_qrbs(t, h, qpu_selected, type_qpu=qpu, shots=100, model='cf') for t, h in zip(Throws, Heights)
]
list_ = [[n, t, h, p["final_score"]] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation)]

pdf = pd.DataFrame(
    list_,
    columns = ["Name", "Throws", "Height", "Final_Score"]
)
#pdf.sort_values(["Final_Score"], ascending=False)
pdf

In [ ]:
for p in player_evaluation:
    plt.plot(p["z"])
plt.legend(list(pdf["Name"]))

In [ ]:
[(i.attribute, i.precision) for i in  player_evaluation[1]["output_facts"]]

In [ ]:
[(i.attribute, i.precision) for i in  player_evaluation[3]["output_facts"]]

In [ ]:
pdf_ = get_pdf_facts(player_evaluation)
pdf_["Throws"] = Throws
pdf_["Heights"] = Heights
pdf_["Name"] = Name
pdf_

We can use different engines for computing the imprecision.

In [ ]:
player_evaluation = [
    basquet_qrbs(t, h, qpu_selected, type_qpu=qpu, shots=0, model='fuzzy') for t, h in zip(Throws, Heights)
]
list_ = [[n, t, h, p["final_score"]] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation)]

pdf = pd.DataFrame(
    list_,
    columns = ["Name", "Throws", "Height", "Final_Score"]
)
pdf.sort_values(["Final_Score"], ascending=False)

In [ ]:
pdf_ = get_pdf_facts(player_evaluation)
pdf_["Throws"] = Throws
pdf_["Heights"] = Heights
pdf_["Name"] = Name
pdf_

In [ ]:
player_evaluation = [
    basquet_qrbs(t, h, qpu_selected, type_qpu=qpu, shots=0, model='bayes') for t, h in zip(Throws, Heights)
]
list_ = [[n, t, h, p["final_score"]] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation)]

pdf = pd.DataFrame(
    list_,
    columns = ["Name", "Throws", "Height", "Final_Score"]
)
pdf.sort_values(["Final_Score"], ascending=False)

In [ ]:
pdf_ = get_pdf_facts(player_evaluation)
pdf_["Throws"] = Throws
pdf_["Heights"] = Heights
pdf_["Name"] = Name
pdf_